# S47 journeys
This notebook looks at S47 journeys and whether they follow the usual trajectory of S47 -> ICPC -> CPP start.
The output of this notebook is table with the columns:
- Source
- Destination
- Count
- Local Authority
- Demographics (gender, age, ethnicity, disability)

This is the shape required to plug the table into a PowerBI Sankey diagram.

In [ ]:
import os
import pandas as pd
import numpy as np

%run "00-config.ipynb"
%load_ext autoreload
%autoreload 2

### Config

#### Filepaths

In [ ]:
input_file = os.path.join(flatfile_folder, 'main_flatcin.csv')
output_file = os.path.join(output_folder, 's47-sankey.csv')

#### Key assumptions

In [ ]:
# End of CIN Census (to calculate age)
cin_census_close = pd.Timestamp(year=2020, month=3, day=31)

# Max days S47 -> CCP for both to be linked
s47_cpp = 60

# Max days ICPC -> CCP for both to be linked
icpc_cpp = 45

# Date from which S47 is too recent to determine next journey
s47_max_date = cin_census_close - pd.Timedelta("60 days") # 60 days before CIN Census closes

# Date from which ICPC is too recent to determine next journey
icpc_max_date = cin_census_close - pd.Timedelta("45 days") # 45 days before CIN Census closes

### Data wrangling

In [ ]:
# Load flatfile
df = pd.read_csv(input_file)

# Only keep 2 subsets: S47 events and CPP start events. 
s47 = df[df.Type == 'S47ActualStartDate']
cpp = df[df.Type == 'CPPstartDate']

# Drop empty cols
s47.dropna(axis=1, how='all', inplace=True)
cpp.dropna(axis=1, how='all', inplace=True)

In [ ]:
# Merge S47 and CPP together, to match each S47 event with CPP events (if they occurred)
data = s47.merge(cpp[['LAchildID', 'LA', 'CPPstartDate']], how='left', on=['LAchildID', 'LA'])
data.head()

In [ ]:
# We might have merged some CPP events that happened to the child, but before the ICPC or much later.
# In that case, the ICPC and the CPP are not related.
# Rule: if the CPP happened before the ICPC, or more than X days later (defined at top of notebook), they are not related.

# Turn relevant columns into dates
data['S47ActualStartDate'] = pd.to_datetime(data['S47ActualStartDate'])
data['DateOfInitialCPC'] = pd.to_datetime(data['DateOfInitialCPC'])
data['CPPstartDate'] = pd.to_datetime(data['CPPstartDate'])

# Calculate icpc_to_cpp: length of time between ICPC and CPP
data['icpc_to_cpp'] = data['CPPstartDate'] - data['DateOfInitialCPC']
data['icpc_to_cpp'] = data['icpc_to_cpp'].dt.days

# Calculate s47_to_cpp: length of time between S47 and CPP
data['s47_to_cpp'] = data['CPPstartDate'] - data['S47ActualStartDate']
data['s47_to_cpp'] = data['s47_to_cpp'].dt.days

data.head()

In [ ]:
# Let's look at the distribution of days between ICPC and CPP start
data.hist(column='icpc_to_cpp')

# We can see that some pairs do not make sense: the CPP happened earlier or much later

In [ ]:
# Only keep rows where:
# CPP start within X days of ICPC (defined top of notebook), or
# CPP start within X days of CPP (defined top of notebook), or
# There are no CPP after the S47

relevant_data = data[((data.icpc_to_cpp >= 0) & (data.icpc_to_cpp <= icpc_cpp)) | 
                     ((data.s47_to_cpp >= 0) & (data.s47_to_cpp <= s47_cpp)) | 
                     data.CPPstartDate.isnull()]
print(relevant_data.shape, data.shape)

### Shaping the PowerBI Sankey diagram input
PowerBI requires a dataset with a Source (start of flow) and Destination (end of flow), with a count of occurrences.
https://powerbi.microsoft.com/en-us/blog/visual-awesomeness-unlocked-sankey-diagram/

We have several Source -> Destination options:
- Step 1:
    - S47 to ICPC
    - S47 to CPP directly
    - S47 to nothing (no ICPC nor CPP)
    - S47 TBD (S47 occurred within 21 days of closing the CIN Census)
- Step 2:
    - ICPC to CPP
    - ICPC to no CPP
    - ICPC TBD (ICPC occurred within 2 months of closing the CIN Census)

We also want to keep the Demographics columns (age, ethnicity, etc.) to be able to filter the sankeys.

In [ ]:
# Generate Source and Destination for step 1

step1 = relevant_data.copy()

# The source is necessarily S47 for step 1
step1['Source'] = 'S47 strategy discussion'

# The destination varies
step1['Destination'] = np.nan # Create empty col
# S47 -> ICPC
step1.loc[step1['DateOfInitialCPC'].notnull(), 'Destination'] = 'ICPC'
# S47 -> CPP directly
step1.loc[step1['DateOfInitialCPC'].isnull() & step1['CPPstartDate'].notnull(), 'Destination'] = 'CPP start'
# S47 -> TBD (too recent)
step1.loc[step1['Destination'].isnull() & (step1['S47ActualStartDate'] >= s47_max_date), 'Destination'] = 'TBD - S47 too recent'
# S47 -> No ICPC
step1.loc[step1['Destination'].isnull(), 'Destination'] = 'No ICPC nor CPP'


# Look at resulting trends
step1.Destination.value_counts(dropna=False)

In [ ]:
# Generate Source and Destination for step 2

# Step 2 starts from all S47 that got to ICPC in step 1
step2 = step1[step1.Destination == 'ICPC']

# The source is necessarily ICPC for step 2
step2['Source'] = 'ICPC'

# The destination varies
step2['Destination'] = np.nan # Empty col
# ICPC -> CPP
step2.loc[step2['CPPstartDate'].notnull(), 'Destination'] = 'CPP start'
# ICPC -> TBD (too recent)
step2.loc[step2['Destination'].isnull() & (step2['DateOfInitialCPC'] >= icpc_max_date), 'Destination'] = 'TBD - ICPC too recent'
# ICPC -> No CPP
step2.loc[step2['Destination'].isnull(), 'Destination'] = 'No CPP'

# Look at resulting trends
step2.Destination.value_counts(dropna=False)

In [ ]:
# Bring Steps 1 & 2 together
s47_journey = pd.concat([step1, step2])

# Calculate age of child during the S47, based on cin_census_close (defined top of notebook) 
s47_journey['Age'] = s47_journey['S47ActualStartDate'].dt.year - s47_journey['PersonBirthDate']

# Group by columns of interest
cols = ['Source', 'Destination', 'Age', 'GenderCurrent', 'Ethnicity', 'Disabilities', 'LA']
s47_sankey = s47_journey.groupby(cols)['LAchildID'].count().reset_index()

# Rename columns for clarity
s47_sankey.rename(columns={"GenderCurrent": "Gender", "LAchildID": "Count"}, inplace=True)

s47_sankey.head()

In [ ]:
# Save
s47_sankey.to_csv(output_file, index=False)